In [125]:
from faker import Faker
import numpy as np
import pandas as pd
import random
from datetime import timedelta, datetime
import math


In [126]:
#inital distribution (fraudCount alr decreased from transCount and Sum)
#most fraud isnt labeled as "electronic", but the electronic is where the majority of the money is stolen, this will be represented by them only getting 100k+ transactions (small chance for noise messing whith this)

preferred_num = {
    "fraudCount": 55331,
    "fraudSum": 8333.0,
    "fraudIsElectronic": 5394,
    "fraudStrongValid": 19426,
    "fraudWeakValid": 33033,
    "fraudDomestic": 5795,
    "fraudInternational": 44142,
    "transCount": 99379358,
    "transSum": 211519426.0,
    "transIsElectronic": 73439211,
    "transStrongValid": 83764064,
    "transWeakValid": 15612772,
    "transDomestic": 99249418,
    "transInternational": 129940,

}

preferred_trans_amount = [
    ((0, 10000), 28333066),
    ((10000, 50000), 33469604),
    ((50000, 100000), 10766020),
    ((100000, 1000000), 21734101),
    ((1000000, 10000000), 4300618),
    ((10000000, 200000000), 825886)
]


#preferred_trans_amount = {
#    "0-10k": 28333066,
#    "10k-50k": 33469604,
#    "50k-100k": 10766020,
#    "100k-1m": 21734101,
#    "1m-10m": 4300618,
#    "10m-": 825886,
#}


generated_num = preferred_num.copy()
generated_trans_amount = preferred_trans_amount.copy()


fake = Faker('hu_HU')


In [127]:
def GenerateAmount():
    range_options, weights = zip(*generated_trans_amount)
    choice = random.choices(
        range_options,
        weights=weights,
        k = 1
    )[0]

    for i, (r, w) in enumerate(generated_trans_amount):
        if r == choice:
            neW = w-1
            generated_trans_amount[i] = (r, neW)

    return round(random.randint(*choice), -1)
    
def GenerateValidation(is_fraud):
    
    if (is_fraud):
        choice = random.choices(
            [0, 1],
            weights=[generated_num['fraudWeakValid'], generated_num['fraudStrongValid']],
            k = 1
        )[0]

        if (choice):
            generated_num['fraudStrongValid'] -= 1
        else:
            generated_num['fraudWeakValid'] -= 1
    
    else:
        choice = random.choices(
            [0, 1],
            weights=[generated_num['transWeakValid'], generated_num['transStrongValid']],
            k = 1
        )[0]

        if (choice):
            generated_num['transStrongValid'] -= 1
        else:
            generated_num['transWeakValid'] -= 1
    
    return choice

def GenerateInternational(is_fraud):
    if (is_fraud):
        choice = random.choices(
            [0, 1],
            weights=[generated_num['fraudDomestic'], generated_num['fraudInternational']],
            k = 1
        )[0]

        if (choice):
            generated_num['fraudInternational'] -= 1
        else:
            generated_num['fraudDomestic'] -= 1
    
    else:
        choice = random.choices(
            [0, 1],
            weights=[generated_num['transDomestic'], generated_num['transInternational']],
            k = 1
        )[0]

        if (choice):
            generated_num['transInternational'] -= 1
        else:
            generated_num['transDomestic'] -= 1
    
    return choice

def GenerateType(is_fraud):
    if (is_fraud):
        choice = random.choices(
            [0, 1],
            weights=[generated_num['fraudCount'] - generated_num['fraudIsElectronic'], generated_num['fraudIsElectronic']],
            k = 1
        )[0]

        if (choice):
            generated_num['fraudIsElectronic'] -= 1
    
    else:
        choice = random.choices(
            [0, 1],
            weights=[generated_num['transCount'] - generated_num['transIsElectronic'], generated_num['transIsElectronic']],
            k = 1
        )[0]

        if (choice):
            generated_num['transIsElectronic'] -= 1
        
    return choice

In [128]:

start_date = datetime(2024, 10, 1, 0, 0, 0, 0)

data = []

for i in range(100000):
    sum = GenerateAmount()
    #print(sum)

    fraud = random.choices([0, 1], weights=[generated_num["transCount"], generated_num["fraudCount"]], k = 1)[0]
    
    valid = GenerateValidation(fraud)

    inter = GenerateInternational(fraud)

    type = GenerateType(fraud)

    #kb 13 tranzakcióként telik el egy másodperc ==> egy nap kb 1123200 tranzakció, lefelé v 76.92 millisec/trans 
    #math.floor(i/1123200)

    start_date = start_date + timedelta(milliseconds=76.92 + random.uniform(-5.0, 5.0))

    if (fraud == 1):
        generated_num["fraudCount"] -= 1
        generated_num["fraudSum"] -= sum/1000000
    else:
        generated_num["transCount"] -= 1
        generated_num["transSum"] -= sum/1000000

    #type: 0 => card, 1 => electronic
    #valid_str: 0 => weak, 1 => strong

    row = {
        'id': i + 1,
        'fraud': fraud,
        'type': type,
        'valid_strength': valid,
        'international': inter,
        'amount': sum,
        'init_name': fake.name(),
        'rec_name': fake.name(),
        'init_account': str(random.randint(10000000, 99999999)) + "-" + str(random.randint(10000000, 99999999)) + "-" + str(random.randint(10000000, 99999999)),
        'rec_account': str(random.randint(10000000, 99999999)) + "-" + str(random.randint(10000000, 99999999)) + "-" + str(random.randint(10000000, 99999999)),
        'date': start_date.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    }
    data.append(row)


df = pd.DataFrame(data)

df.to_csv("./Data/GeneratedDataset.csv", index = False, encoding = 'utf-8')

print(f"{generated_num['fraudCount']} {generated_num['fraudSum']}\n{generated_num['transCount']} {generated_num['transSum']}")
    

55293 8314.385420000004
99279396 211396070.15474117
